### Py notebook to receive OpenSignals JSON data and forward as OSC

In [ ]:
import socket
import json
import ast
import threading
import select
import queue
import pandas as pd
import numpy as np

In [ ]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import time

In [ ]:
class SaveAcquisition(object):
    def __init__(self):
        self.fileTxt = None
        self.hasHeader = False
 
    def start(self):
        self.fileTxt = open("tcp_py3_Acquisition.txt", "w")
 
    def addData(self, data):
        self.fileTxt.write(data)
        self.hasHeader = True
 
    def stop(self):
        self.fileTxt.close()
 
    def getHasHeader(self):
        return self.hasHeader
 

In [ ]:
from pythonosc.udp_client import SimpleUDPClient
osc_ip = "192.168.1.85"
osc_port = 8000

In [ ]:
class DataBuffer(object):
    def __init__(self):
        self.data = []
        self.isFull = False
        self.size = 5000
        self.osc_ip = ""
        self.osc_port = 8000
        self.osc_client = SimpleUDPClient(self.osc_ip, self.osc_port)  # Create client

dbuff = DataBuffer()

In [ ]:
class TCPClient(object):
    def __init__(self):
        # self.tcpIp = ''
        self.tcpPort = 3333
        self.tcpIp = 'localhost'
 
        self.buffer_size = 99999
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.inputCheck = []
        self.outputCheck = []
        self.isChecking = False
        self.isAcquiring = False
        self.msgQueue = queue.Queue()
 
        self.txtFile = SaveAcquisition()
 
    def connect(self):
        self.socket.connect((self.tcpIp, self.tcpPort))
        self.outputCheck.append(self.socket)
        self.isChecking = True
 
    def start(self):
        print("XX ", self.isChecking)
        thread = threading.Thread(target=self.msgChecker)
        thread.daemon = True
        thread.start()
 
    def stop(self):
        self.isChecking = False
        self.socket.close()
 
    def msgChecker(self):
        global dbuff
        while self.isChecking:
            readable, writable, exceptional = select.select(self.inputCheck, self.outputCheck, self.inputCheck)
            
            
            for s in readable:
                message = s.recv(self.buffer_size)
                if not self.isAcquiring:
                    #print(message)
                    self.inputCheck = []
                else:
                    #print(message)
                    message = json.loads(message.decode('utf8'))
                    message = message["returnData"]
                    if not self.txtFile.getHasHeader():
                        newLine = json.dumps(message) + "\n"
                        self.txtFile.addData(newLine)
                    else:
                        dataframe = []
                        for device in list(message.keys()):
                            dataframe.append(pd.DataFrame(message[device]))
                        dataframe = pd.concat(dataframe, axis=1, ignore_index=True)
                        for line in dataframe.values:
                            self.txtFile.addData('\n')
                            self.txtFile.addData(",".join([str(x) for x in line]))
                           
                            #TODO Rolling list up to the defined size
                            if not dbuff.isFull:
                                if len(dbuff.data) < dbuff.size:
                                    dbuff.data.append([float(x) for x in line])
                                else:
                                    dbuff.isFull = True
                                    dbuff.data.append(dbuff.data.pop(0))
                                    datapoint = ([float(x) for x in line])
                                    dbuff.data[-1] = datapoint
                                    dbuff.osc_client.send_message("/oscpy/", datapoint)

                                
                            else:
                                dbuff.data.append(dbuff.data.pop(0))
                                datapoint = ([float(x) for x in line])
                                dbuff.data[-1] = datapoint
                                dbuff.osc_client.send_message("/oscpy/", datapoint)
 
            for s in writable:
                try:
                    next_msg = self.msgQueue.get_nowait()
                except queue.Empty:
                    pass
                else:
                    print("send ")
                    self.socket.send(next_msg.encode('utf-8'))
 
            for s in exceptional:
                print("exceptional ", s)
 
    def addMsgToSend(self, data):
        self.msgQueue.put(data)
        if self.socket not in self.outputCheck:
            self.outputCheck.append(self.socket)
        if self.socket not in self.inputCheck:
            self.inputCheck.append(self.socket)
 
    def setIsAcquiring(self, isAcquiring):
        self.isAcquiring = isAcquiring
        if self.isAcquiring:
            self.txtFile = SaveAcquisition()
            self.txtFile.start()
        else:
            self.txtFile.stop()
            
    def collectBuffer(self, isAcquiring):
        if self.isAcquiring:
            self.txtFile = SaveAcquisition()
            self.txtFile.start()
        else:
            self.txtFile.stop()

In [ ]:
CONNECTION = TCPClient()
CONNECTION.connect()
t0 = time.time()


In [ ]:
CONNECTION.setIsAcquiring(True)
new_msg = 'start'
CONNECTION.start()
CONNECTION.addMsgToSend(new_msg)

In [ ]:
new_msg = 'stop'
CONNECTION.setIsAcquiring(False)
CONNECTION.addMsgToSend(new_msg)

In [ ]:
dbuff.data

In [ ]:
CONNECTION.stop()